In [1]:
# This file serves as a starting template for other scripts.
import glob
import os
import sys
import random
import time
import numpy as np
import cv2
import open3d as o3d    # Open3D is used for 3D visualization of point clouds
from matplotlib import cm   # Colormap for point cloud visualization
import math

# Add the carla egg file to the python path
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

client = carla.Client('localhost', 2000)
# client.reload_world()
world = client.get_world()
blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
random_start_point = random.choice(spawn_points)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create Actor List
# Function to clean up all actors
def cleanUp():
    for actor in actor_list:
        actor.destroy()
    print("All cleaned up!")

actor_list = []

In [3]:
# Spawn vehicle
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
# vehicle = world.spawn_actor(vehicle_bp, random_start_point)
vehicle = world.spawn_actor(vehicle_bp, start_point)
actor_list.append(vehicle)

In [4]:
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:, :3]

def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]
    ]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]))
    vis.add_geometry(axis)

C:\Users\thoma\AppData\Local\Temp\ipykernel_24168\3537187559.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  VIRIDIS = np.array(cm.get_cmap('plasma').colors)
C:\Users\thoma\AppData\Local\Temp\ipykernel_24168\3537187559.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))


In [5]:
def lidar_callback(point_cloud, point_list):
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    # 4 columns: x, y, z, intensity (distance from detector)
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate intensity and compute color, color based on distance
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_ [
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])
    ]
 
    # form a point cloud
    points = data[:, :-1]
    # reverse y axis to make unreal engine y axis match normal y axis format
    points[:, :1] = -points[:, :1]

    # Store points and colors in open3d point cloud
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def radar_callback(data, point_list):
    radar_data = np.zeros((len(data), 4))

    for i, detection in enumerate(data):
        x = detection.depth = math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth = math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth = math.sin(detection.altitude)

        radar_data[i, :] = [x, y, z, detection.velocity]

    intensity = np.abs(radar_data[:, -1])
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_ [
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])
    ]

    points = radar_data[:, :-1]
    points[:, :1] = -points[:, :1]
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [6]:
# Set up Lidar and radar sensors
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('rotation_frequency', '20')
lidar_bp.set_attribute('points_per_second', '320000')
lidar_bp.set_attribute('channels', '64')
lidar_bp.set_attribute('upper_fov', '15')
lidar_bp.set_attribute('lower_fov', '-25')
# lidar_bp.set_attribute('sensor_tick', '0.05')
# lidar_bp.set_attribute('noise_stdev', '0.1')

lidar_init_transform = carla.Transform(carla.Location(z=2.0))
lidar = world.spawn_actor(lidar_bp, lidar_init_transform, attach_to=vehicle)
actor_list.append(lidar)

# Set up radar sensor
radar_bp = blueprint_library.find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('range', '100')
radar_bp.set_attribute('points_per_second', '10000')
# radar_bp.set_attribute('sensor_tick', '0.05')
# radar_bp.set_attribute('noise_stdev', '0.1')
radar_init_transform = carla.Transform(carla.Location(z=2.0))
radar = world.spawn_actor(radar_bp, radar_init_transform, attach_to=vehicle)
actor_list.append(radar)

# Set up camera sensor
camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')
camera_init_transform = carla.Transform(carla.Location(z=1.6, x=0.4))
camera = world.spawn_actor(camera_bp, camera_init_transform, attach_to=vehicle)
actor_list.append(camera)

# Set up point cloud
point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

# Set up visualization
image_width = camera_bp.get_attribute('image_size_x').as_int()
image_height = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_height, image_width, 4))}

lidar.listen(lambda data: lidar_callback(data, point_list))
radar.listen(lambda data: radar_callback(data, radar_list))
camera.listen(lambda data: camera_callback(data, camera_data))




In [7]:
# Display data
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Carla Lidar', width=800, height=600, left=480, top=270)
vis.get_render_option().point_size = 1.0
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

frame = 0
while True:
    if frame ==2: 
        vis.add_geometry(point_list)
        vis.add_geometry(radar_list)
    vis.update_geometry(point_list)
    vis.update_geometry(radar_list)

    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.005)   # fix Open3D jitter issue
    frame += 1

    cv2.imshow('RGB Camera', camera_data['image'])

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
radar.stop()
lidar.stop()
camera.stop()
cleanUp()
vis.destroy_window()

C:\Users\thoma\AppData\Local\Temp\ipykernel_24168\3804114712.py:35: RuntimeWarning: divide by zero encountered in log
  intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))


All cleaned up!
